In [1]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Import required modules
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
)
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain_core.runnables import RunnableLambda, RunnableMap
from langchain_core.output_parsers import StrOutputParser
from langchain.load import dumps, loads
from langchain_community.chat_models import ChatOllama
from fastapi.responses import StreamingResponse


from tqdm import tqdm
import torch
import re
import os

import gradio as gr

Embedding Config

In [2]:
# Embed the text content in split_docs
embedding_model = HuggingFaceEmbeddings(
    model_name="intfloat/e5-base-v2",
    model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"},
    encode_kwargs={"normalize_embeddings": True},
)

/tmp/ipykernel_3065683/542832890.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/home/sfnavas-f/Projects/DomainMind/.venv/lib/python3.13/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


Load Existing Vector DB

In [3]:
# Load the vector store
vectorstore = Chroma(
    persist_directory="../data/doc_vectordb",
    embedding_function=embedding_model,
)

/tmp/ipykernel_3065683/360242083.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


Retriever Config

In [4]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

Setting up Local LLM with Ollama. 

In [5]:
llm = ChatOllama(model="qwen3:8b", base_url="http://localhost:11434", streaming=True)
# llm = ChatOllama(model="gemma3:4b", base_url="http://localhost:11434", streaming=True)

/tmp/ipykernel_3065683/3811996168.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="qwen3:8b", base_url="http://localhost:11434", streaming=True)


MultiQuery Generation Chain.

In [6]:
# Prompt
template = """You are an AI scientific research assistant. Your task is to generate two 
different versions of the given user question and the conversation history so as to retrieve relevant documents from a vector 
database. The vector database consists of scientific papers, articles, books and other academic 
resources related to the field of Statistical Physics, Computational Physics, Statistics, Soft-Matter Physics 
and other related fields. By generating multiple perspectives on the user question, your goal is to help 
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. If the below question is unrelated or does not require 
additional context, you can respond with "No relevant questions found."
Original question: {question}
Conversation history: {history}"""
multi_prompt = ChatPromptTemplate.from_template(template)


def remove_think_blocks(text):
    # Remove all <think>...</think> blocks from the text
    return re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL).strip()


# Query generation cHAIN
generate_query_chain = (
    multi_prompt
    | llm
    | StrOutputParser()
    | remove_think_blocks
    | (lambda x: x.split("\n"))
)

Retrieval Chain for multiple queries.

In [7]:
# Function to make a single list of the retrieved documents
def get_unique_union(documents: list[list]):
    """Unique union of retrieved docs"""
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]


# Retrieval Chain
retrieval_chain = generate_query_chain | retriever.map() | get_unique_union

# Summary Chain 

In [8]:
# Prompt
template = """
You are an AI scientific research assistant. Summarize the following conversation between a user and an assistant.
- Focus on the key points, main ideas, and any important questions and answers.
- Exclude irrelevant, repetitive, or off-topic content.
- Use your own words; do not copy the conversation verbatim.
- Keep the summary concise and within 512-1024 tokens.
- Format the summary as a narration of how the conversation between the user and the assistant unfolded.
- If the conversation is empty, respond with "No conversation history."

Original conversation:
{history}
"""
summarization_prompt = ChatPromptTemplate.from_template(template)

# Summarization chain
summarization_chain = (
    summarization_prompt | llm | StrOutputParser() | remove_think_blocks
)

Testing the working of the summarization chain. 

In [9]:
# Example conversation history
history = [
    HumanMessage(content="What is self-aggregation?"),
    AIMessage(
        content="Self-aggregation is the process by which molecules or particles spontaneously organize into ordered structures."
    ),
    HumanMessage(content="Why is it important in materials science?"),
    AIMessage(
        content="It's important because it can lead to new material properties and functionalities."
    ),
]

In [10]:
# See what summary is produced
summary = summarization_chain.invoke({"history": history})
print("Summary:", summary)

Summary: The user initiated a conversation by asking about “self-aggregation.” The AI assistant responded by defining self-aggregation as the spontaneous organization of molecules or particles into ordered structures. Following this explanation, the user then inquired about the significance of self-aggregation within materials science. The AI replied that it's important due to its potential to create materials with novel properties and functionalities. Essentially, the conversation focused on a definition of self-aggregation and its relevance to the development of new materials.


In [ ]:
# Can be used to print the prompt  within the RAG chain
def debug_prompt(messages):
    print("Prompt to LLM:")
    for m in messages:
        print(f"{type(m).__name__}: {getattr(m, 'content', m)}")
    return messages


# | RunnableLambda(debug_prompt) Insert this right after the prompt prep and right before the llm call

# RAG Chain 

Prompt 

In [9]:
messages = [
    SystemMessage(
        content="""
        You are a helpful scientific assistant. You need to answer the question in a scientifically sound manner, \\
        combining your own knowledge and the  provided context and the conversation history. If you use information \\
        from the provided context, cite it. If the answer is based on your own knowledge, state so. The conversation \\
        history is provided above. You also have further context regarding the current question below from snippets \\
        from various scientific papers. 
        """
    ),
    HumanMessagePromptTemplate.from_template(
        "Context:\n{context}\n\nQuestion: {question}\n\n History: {history}"
    ),
]

prompt = ChatPromptTemplate.from_messages(messages)


rag_chain_multi_query = (
    {
        "history": RunnableLambda(
            lambda x: summarization_chain.invoke({"history": x["history"]})
        ),
        "context": RunnableLambda(
            lambda x: retrieval_chain.invoke(
                {
                    "question": x["question"],
                    "history": summarization_chain.invoke({"history": x["history"]}),
                }
            )
        ),
        "question": RunnableLambda(lambda x: (x["question"])),
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
def split_think_and_answer(response):
    """Extracts text after </think>."""
    match = re.search(r"</think>\s*(.*)", response, re.DOTALL)
    if match:
        return match.group(1).strip()
    return response.strip()


def get_thought(response):
    """Extracts text inside <think>...</think>."""
    match = re.search(r"<think>(.*?)</think>", response, re.DOTALL)
    if match:
        return match.group(1).strip()
    return None


def remove_details_blocks(text):
    # Remove all <details>...</details> blocks, including nested tags and multiline content
    return re.sub(r"<details[\s\S]*?</details>", "", text, flags=re.DOTALL).strip()


def rag_qa_multi_query(message, history):
    # Convert Gradio history format to LangChain message format
    history_langchain_format = []
    # Convert Gradio history (list of {"role": ..., "content": ...}) to LangChain format
    for turn in history:
        if turn["role"] == "user":
            history_langchain_format.append(HumanMessage(content=turn["content"]))
        elif turn["role"] == "assistant":
            cleaned_content = remove_details_blocks(turn["content"])
            history_langchain_format.append(AIMessage(content=cleaned_content))
    output = ""
    try:
        for chunk in rag_chain_multi_query.stream(
            {"history": history_langchain_format, "question": message}
        ):
            output += chunk
            yield {"role": "assistant", "content": output}

        # Pass both current message and history to the chain
        # response = rag_chain_multi_query.invoke(
        #     {"history": history_langchain_format, "question": message}
        # )
        # thought = get_thought(response)
        # answer = split_think_and_answer(response)

        # if thought:
        #     # Add collapsible section with the <think> content
        #     final_answer = f"<details><summary><b>🤔 Thinking</b></summary><pre>{thought}</pre></details>\n\n{answer}"
        # else:
        #     final_answer = answer

        # return {"role": "assistant", "content": final_answer}
    except Exception as e:
        return {"role": "assistant", "content": f"❌ Error: {str(e)}"}


# Create and launch the chat interface with memory
demo = gr.ChatInterface(
    fn=rag_qa_multi_query,
    type="messages",
    title="📄 Scientific PDF Chatbot",
    description="Ask questions about your scientific PDFs. Powered by RAG + Qwen3:8B",
    examples=["What are colloidal particles?", "Tell me more about that"],
)

demo.launch(share=False, inline=False, inbrowser=False)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


/home/sfnavas-f/Projects/DomainMind/.venv/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/tmp/ipykernel_3045617/3087458109.py:9: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return [loads(doc) for doc in unique_docs]


In [10]:
def remove_details_blocks(text):
    # Remove all <details>...</details> blocks, including nested tags and multiline content
    return re.sub(r"<details[\s\S]*?</details>", "", text, flags=re.DOTALL).strip()


def rag_qa_multi_query(message, history):
    # Convert Gradio history format to LangChain message format
    history_langchain_format = []
    # Convert Gradio history (list of {"role": ..., "content": ...}) to LangChain format
    for turn in history:
        if turn["role"] == "user":
            history_langchain_format.append(HumanMessage(content=turn["content"]))
        elif turn["role"] == "assistant":
            cleaned_content = remove_details_blocks(turn["content"])
            history_langchain_format.append(AIMessage(content=cleaned_content))
    try:
        raw_output = ""
        processed_full_output = ""

        # Stream the response per token and process <think> tags
        # for thinking models
        for chunk in rag_chain_multi_query.stream(
            {"history": history, "question": message}
        ):
            raw_output += chunk

            # Check for <think> start
            if "<think>" in chunk:
                chunk = chunk.replace("<think>", "Reasoning:")

            if "</think>" in chunk:
                chunk = chunk.replace("</think>", "End of Reasoning\n")

            processed_full_output += chunk

            yield {
                "role": "assistant",
                "content": processed_full_output,
            }

    except Exception as e:
        return {"role": "assistant", "content": f"❌ Error: {str(e)}"}


# Create and launch the chat interface with memory
demo = gr.ChatInterface(
    fn=rag_qa_multi_query,
    type="messages",
    title="📄 Scientific PDF Chatbot",
    description="Ask questions about your scientific PDFs. Powered by RAG + Qwen3:8B",
    examples=["What are colloidal particles?", "Tell me more about that"],
)

demo.launch(share=False, inline=False, inbrowser=False)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


/home/sfnavas-f/Projects/DomainMind/.venv/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/tmp/ipykernel_3065683/3087458109.py:9: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return [loads(doc) for doc in unique_docs]
/home/sfnavas-f/Projects/DomainMind/.venv/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


When streaming content from Gradio, < tags > like this can cause issues. If we do not stream and print out the full content at the end, it works fine. Else, when streaming after the first < tag >, the rest of the content is not printed.